# Write your own GNN module

https://docs.dgl.ai/en/0.9.x/tutorials/blitz/3_message_passing.html#sphx-glr-tutorials-blitz-3-message-passing-py

In [1]:
import os

os.environ["DGLBACKEND"] = "pytorch"

In [2]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F

## Message passing and GNNs

In [3]:
import dgl.function as fn

class SAGEConv(nn.Module):
    """Graph convolution module used by the GraphSAGE model.

    Parameters
    ----------
    in_feat : int
        Input feature size.
    out_feat : int
        Output feature size.
    """
    def __init__(self, in_feat, out_feat):
        super(SAGEConv, self).__init__()
        # A linear submodule for projecting the input and neighbor feature to the output.
        self.linear = nn.Linear(in_feat * 2, out_feat)

    def forward(self, g, h):
        """Forward computation

        Parameters
        ----------
        g : Graph
            The input graph.
        h : Tensor
            The input node feature.
        """
        with g.local_scope():
            g.ndata['h'] = h
            # update_all is a message passing API.
            g.update_all(message_func=fn.copy_u('h', 'm'), reduce_func=fn.mean('m', 'h_N'))
            h_N = g.ndata['h_N']
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)

In [4]:
class Model(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(Model, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats)
        self.conv2 = SAGEConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

In [5]:
import dgl.data

dataset = dgl.data.CoraGraphDataset()
g = dataset[0]

def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    all_logits = []
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(200):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that we should only compute the losses of the nodes in the training set,
        # i.e. with train_mask 1.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        all_logits.append(logits.detach())

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))

model = Model(g.ndata['feat'].shape[1], 16, dataset.num_classes)
train(g, model)

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
In epoch 0, loss: 1.952, val acc: 0.122 (best 0.122), test acc: 0.130 (best 0.130)
In epoch 5, loss: 1.884, val acc: 0.194 (best 0.194), test acc: 0.199 (best 0.199)
In epoch 10, loss: 1.752, val acc: 0.400 (best 0.400), test acc: 0.391 (best 0.391)
In epoch 15, loss: 1.547, val acc: 0.486 (best 0.486), test acc: 0.488 (best 0.488)
In epoch 20, loss: 1.274, val acc: 0.562 (best 0.562), test acc: 0.573 (best 0.573)
In epoch 25, loss: 0.964, val acc: 0.638 (best 0.638), test acc: 0.634 (best 0.634)
In epoch 30, loss: 0.663, val acc: 0.692 (best 0.692), test acc: 0.697 (best 0.697)
In epoch 35, loss: 0.416, val acc: 0.712 (best 0.712), test acc: 0.725 (best 0.721)
In epoch 40, loss: 0.245, val acc: 0.728 (best 0.728), test acc: 0.743 (best 0.743)
In epoch 45, loss: 0.143, val acc: 0.736 (best 0.736), test acc:

In [6]:
g.ndata

{'feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), 'label': tensor([3, 4, 4,  ..., 3, 3, 3]), 'val_mask': tensor([False, False, False,  ..., False, False, False]), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True]), 'train_mask': tensor([ True,  True,  True,  ..., False, False, False])}

## More customization

In [7]:
class WeightedSAGEConv(nn.Module):
    """Graph convolution module used by the GraphSAGE model with edge weights.

    Parameters
    ----------
    in_feat : int
        Input feature size.
    out_feat : int
        Output feature size.
    """
    def __init__(self, in_feat, out_feat):
        super(WeightedSAGEConv, self).__init__()
        # A linear submodule for projecting the input and neighbor feature to the output.
        self.linear = nn.Linear(in_feat * 2, out_feat)

    def forward(self, g, h, w):
        """Forward computation

        Parameters
        ----------
        g : Graph
            The input graph.
        h : Tensor
            The input node feature.
        w : Tensor
            The edge weight.
        """
        with g.local_scope():
            g.ndata['h'] = h
            g.edata['w'] = w
            g.update_all(message_func=fn.u_mul_e('h', 'w', 'm'), reduce_func=fn.mean('m', 'h_N'))
            h_N = g.ndata['h_N']
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)

In [8]:
class Model(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(Model, self).__init__()
        self.conv1 = WeightedSAGEConv(in_feats, h_feats)
        self.conv2 = WeightedSAGEConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat, torch.ones(g.num_edges(), 1).to(g.device))
        h = F.relu(h)
        h = self.conv2(g, h, torch.ones(g.num_edges(), 1).to(g.device))
        return h

model = Model(g.ndata['feat'].shape[1], 16, dataset.num_classes)
train(g, model)

In epoch 0, loss: 1.951, val acc: 0.316 (best 0.316), test acc: 0.319 (best 0.319)
In epoch 5, loss: 1.871, val acc: 0.320 (best 0.320), test acc: 0.326 (best 0.326)
In epoch 10, loss: 1.721, val acc: 0.538 (best 0.580), test acc: 0.525 (best 0.576)
In epoch 15, loss: 1.503, val acc: 0.512 (best 0.580), test acc: 0.493 (best 0.576)
In epoch 20, loss: 1.230, val acc: 0.568 (best 0.580), test acc: 0.531 (best 0.576)
In epoch 25, loss: 0.937, val acc: 0.622 (best 0.622), test acc: 0.595 (best 0.595)
In epoch 30, loss: 0.660, val acc: 0.698 (best 0.698), test acc: 0.683 (best 0.683)
In epoch 35, loss: 0.429, val acc: 0.728 (best 0.728), test acc: 0.732 (best 0.732)
In epoch 40, loss: 0.262, val acc: 0.746 (best 0.746), test acc: 0.756 (best 0.756)
In epoch 45, loss: 0.156, val acc: 0.756 (best 0.760), test acc: 0.763 (best 0.763)
In epoch 50, loss: 0.095, val acc: 0.754 (best 0.760), test acc: 0.762 (best 0.763)
In epoch 55, loss: 0.060, val acc: 0.754 (best 0.760), test acc: 0.763 (best 0